In [ ]:
import sys
import itertools
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent))
from os.path import join, basename

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from bulletin import default_input, default_output, data_comeco_pandemia, menor_data_nascimento, hoje
from bulletin.systems import Notifica, CasosConfirmados
from bulletin.services import Metabase
from bulletin.utils import utils, normalize
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols

import random
import gc

In [ ]:
mb = Metabase()
# mb.download_tables()
# mb.notificacao_information_schema(True)

In [ ]:
notifica = Notifica('notifica')
notifica.load(compress=False)

In [ ]:
def converters(self):
    for col in self.schema.loc[self.schema['converters'].notna(),'column']:
        if col in self.df.columns:
            self.df[col] = self.df[col].apply(self.converters[col])

def normalize_dates(self,format='%d/%m/%Y',errors='coerce'):
    for col in self.schema.loc[(self.schema['dtypes']=='datetime64[ns]'),'column']:
        if col in self.df.columns:
            self.df[col] = pd.to_datetime(self.df[col],format=format,errors=errors)
            
            if col == 'data_nascimento':
                self.df.loc[(self.df[col]<menor_data_nascimento)|(self.df[col]>hoje), col] = pd.NaT
            else:
                self.df.loc[(self.df[col]<data_comeco_pandemia)|(self.df[col]>hoje), col] = pd.NaT

def fillna(self):
    # Normalize valores null em campos não nullables
    for col, dtype, fillna in self.schema.loc[self.schema['fillna'].notna(),['column', 'dtypes','fillna']].itertuples(False):
        if col in self.df.columns:
            self.df.loc[self.df[col].isna(),col] = fillna
            self.df[col] = self.df[col].astype(eval(dtype))
            

def fill_category(self):
    for col, dtype, ftable, fkey, fillna in self.schema.loc[self.schema['foreign_table_name'].notna() & self.schema['fillna'].notna(),['column', 'dtypes', 'foreign_table_name','foreign_column_name','fillna']].itertuples(False):
        if (col in self.df.columns):
            categorias = self.tables[ftable][fkey].values
            for categoria_errada in [ valor for valor in self.df[col].unique() if not valor in categorias ]:
                self.df.loc[self.df[col]==categoria_errada,col] = fillna
                self.df[col] = self.df[col].astype(eval(dtype))

fill_category(notifica)

In [ ]:
#         # Normalize int
#         for col, dtype in self.schema.loc[self.schema['dtypes']=='int',['column', 'dtypes']].itertuples(False):
#             if (col in self.df.columns) and (self.df[col].dtype == np.float64):
#                 with Timer(init_msg=f"normalize int in {col}"):
#                     self.df[col] = self.df[col].apply(lambda x: normalize_number(x,fill=-2))
#                     try:
#                         print(f"Trynna type {col} to {dtype}")
#                         self.df[col] = self.df[col].astype(eval(dtype))
#                     except:
#                         print(f"Não deu :/")
#                         pass
                    
#         # Normalize valores categóticos fora das categorias :/ 
#         print(f"\n\naaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\n\n")

#         for col, dtype in self.schema.loc[self.schema['foreign_table_name'].notna(),['column', 'dtypes']].itertuples(False):
#             if (col in self.df.columns) and (self.schema.loc[self.schema['column']==col,'fillna'].notna().values[0]) and (self.df[col].dtype == np.float64):
#                 with Timer(init_msg=f"normalize categoria_errada in {col}"):
#                     categorias = self.tables[self.schema.loc[self.schema['column']==col,'foreign_table_name'].values[0]][self.schema.loc[self.schema['column']==col,'foreign_column_name']].T.values[0].tolist()
#                     print('categorias: ',categorias)
#                     for categoria_errada in [ valor for valor in self.df[col].unique() if not valor in categorias + [-1] ]:
#                         print('categoria_errada: ',categoria_errada, 'fillna: ',self.schema.loc[self.schema['column']==col,'fillna'].values[0])
#                         self.df.loc[self.df[col]==categoria_errada,col] = self.schema.loc[self.schema['column']==col,'fillna'].values[0]
#                     try:
#                         print(f"Trynna type {col} to {dtype}")
#                         self.df[col] = self.df[col].astype(eval(dtype))
#                     except:
#                         print(f"Não deu :/")
#                         pass
        
#         print(f"\n\naaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\n\n")

#         #normalize string types
#         for col in self.schema.loc[self.schema['dtypes']=='str','column']:
#             if (col in self.df.columns):
#                 with Timer(init_msg=f"normalize string in {col}"):
#                     if (self.df[col].dtype == np.float64):
#                         print(f"{col} is float")
#                         self.df.loc[self.df[col].notna(),col] = self.df.loc[self.df[col].notna(),col].apply(lambda x: f'{x:.0f}')
#                     else:
#                         print(f"{col} not is float")
#                         self.df.loc[self.df[col].notna(),col] = self.df.loc[self.df[col].notna(),col].apply(str)


#         # Seleciona melhor data de diagnóstico dentre as datas validas na ordem de prioridade: 
#         # data_coleta -> data_recebimento -> data_liberacao -> data_notificacao
#         with Timer(init_msg="Seleciona melhor data de diagnóstico dentre as datas validas"):
#             self.df['data_diagnostico'] = self.df['data_notificacao']
#             self.df.loc[self.df['data_liberacao'].notnull(), 'data_diagnostico'] = self.df.loc[self.df['data_liberacao'].notnull(), 'data_liberacao']
#             self.df.loc[self.df['data_recebimento'].notnull(), 'data_diagnostico'] = self.df.loc[self.df['data_recebimento'].notnull(), 'data_recebimento']
#             self.df.loc[self.df['data_coleta'].notnull(), 'data_diagnostico'] = self.df.loc[self.df['data_coleta'].notnull(), 'data_coleta']

#         # Calcula a idade com base na data de notificacao, já que no Notifica atualiza quando mexem na ficha '-'
#         with Timer(init_msg="Calcula a idade com base na data de notificacao"):
#             self.df.loc[(self.df['data_nascimento'].notnull()) & (self.df['data_notificacao'].notnull()), 'idade'] = (
#                 self.df.loc[(self.df['data_nascimento'].notnull()) & (self.df['data_notificacao'].notnull())].apply(
#                         lambda row: row['data_notificacao'].year - row['data_nascimento'].year - (
#                                 (row['data_notificacao'].month, row['data_notificacao'].day) <
#                                 (row['data_nascimento'].month, row['data_nascimento'].day)
#                         ), axis=1
#                 )
#             )

#         self.df.loc[self.df['evolucao']==5,'evolucao'] = 2

#         self.hashes()
#         self.check_duplicates()